In [ ]:
import pandas as pd
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import re


In [ ]:
df = pd.read_excel('C:\\Users\\hasan\\OneDrive\\Documents\\Hasan_School_Job_Docs\\Class\\DS for Algo Marketing\\Week2\\Assignment-2_Data.xlsx')


In [ ]:
df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Data Cleaning Process:

- We remove rows where price is lower than or equal to 0.
    - We are making the assumption that:
        1. price lower than 0 are refunded transactions
        2. price equal to 0 are items gifted for free during promotion (which are not actual items the customers would have purchase alongside their "basket"
        
- We also remove any row from the BillNo column that contains the letter 'A'. The Letter A marked BillNo rows corresponding itemname is 'Adjust bad debt'. We also removed rows where BillNo column has missing values (NaN)

- We have identified a lot of CustomerID with NULL values, but have made the decision to include them. This is because we are making the assumption that they are "guest" customers and does not have an account with us.

- We cleaned Itemname column by removing any leading or trailing whitespace characters from the 'Itemname' column in the DataFrame.


In [ ]:
df = df[df['Price']>0]

In [ ]:
df['Itemname'] = df['Itemname'].str.strip()
df.dropna(axis=0, subset=['BillNo'], inplace=True)
df['BillNo'] = df['BillNo'].astype('str')
df = df[~df['BillNo'].str.contains('A')]



basket = (df.groupby(['BillNo', 'Itemname'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('BillNo'))

basket.head()

Itemname,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



- We use the apriori() function from the mlxtend library to find frequent item sets in the binary-encoded basket_sets
- The min_support parameter specifies the minimum support threshold for an itemset to be considered frequent, We tried multiple values for minimum support, the value that gave us the most usable data was 0.0220. We observed that this value strikes a balance between being too general and too specific. It's high enough to filter out infrequent itemsets that may not be relevant for analysis but low enough to capture meaningful patterns in the data.
- We use the association_rules() function to generate association rules from the frequent itemsets with specified metric, "lift", and minimum threshold = 1.


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units).astype(bool)
frequent_itemsets = apriori(basket_sets, min_support=0.0220, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(PACK OF 72 RETROSPOT CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.065395,0.040802,0.022242,0.340109,8.335665,0.019573,1.453572,0.941610
1,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETROSPOT CAKE CASES),0.040802,0.065395,0.022242,0.545113,8.335665,0.019573,2.054586,0.917468
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.052510,0.049443,0.032365,0.616358,12.466118,0.029769,2.477722,0.970758
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.049443,0.052510,0.032365,0.654602,12.466118,0.029769,2.743181,0.967625
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.052510,0.038808,0.023213,0.442064,11.391163,0.021175,1.722765,0.962768


- We used the lift value to be greater than or equal to 1 because Lift values less than 1 indicate that the items occur together less frequently than expected if they were independent. Such rules may not be useful for practical purposes as they represent weak or negative associations. By setting a minimum lift threshold of at least 1, you filter out these uninteresting rules and focus on stronger associations.
- We have also chosen to filter out confidence >= 0.6. This is the highest confidence level we can set to in order for us to be able to retrieve at least 25 association rules.

# 25 Association Rules:


In [ ]:
print ('25 Association Rules')
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.6) ].sort_values(['lift','confidence'], ascending = False).head(25).reset_index(drop = True)

25 Association Rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",(PINK REGENCY TEACUP AND SAUCER),0.037325,0.037581,0.026281,0.704110,18.736021,0.024878,3.252621,0.983330
1,(PINK REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",0.037581,0.037325,0.026281,0.699320,18.736021,0.024878,3.201657,0.983591
2,"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.029093,0.049749,0.026281,0.903339,18.157768,0.024833,9.830774,0.973242
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.049749,0.037581,0.030883,0.620761,16.518142,0.029013,2.537762,0.988645
4,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037581,0.049749,0.030883,0.821769,16.518142,0.029013,5.331558,0.976145
5,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.030883,0.051744,0.026281,0.850993,16.446372,0.024683,6.363855,0.969125
6,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.038552,0.046273,0.027815,0.721485,15.592057,0.026031,3.424335,0.973391
7,(GARDENERS KNEELING PAD KEEP CALM),(GARDENERS KNEELING PAD CUP OF TEA),0.046273,0.038552,0.027815,0.601105,15.592057,0.026031,2.410278,0.981271
8,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.037581,0.051744,0.029093,0.774150,14.961284,0.027148,4.198605,0.969599
9,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.049749,0.051744,0.037325,0.750257,14.499531,0.034751,3.796928,0.979776


# Top 25 Frequent Item sets (Bundles of at least size 2 ), based on relative support

As we are interested only on bundles of at least size 2, we filtered out the itemsets which only contain 1 item.

In [ ]:
print( '25 frequent items')
filtered_frequent_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) >= 2)]
filtered_frequent_itemsets.sort_values('support', ascending = False).head(25).reset_index(drop = True)

25 frequent items


,support,itemsets
0,0.041875,"(JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROS..."
1,0.037325,"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC..."
2,0.036865,"(JUMBO STORAGE BAG SUKI, JUMBO BAG RED RETROSPOT)"
3,0.034564,"(JUMBO BAG RED RETROSPOT, JUMBO SHOPPER VINTAG..."
4,0.032365,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK..."
5,0.032314,"(LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROS..."
6,0.030883,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY..."
7,0.030320,"(LUNCH BAG RED RETROSPOT, LUNCH BAG PINK POLKA..."
8,0.029655,"(JUMBO BAG BAROQUE BLACK WHITE, JUMBO BAG RED..."
9,0.029297,"(LUNCH BAG RED RETROSPOT, JUMBO BAG RED RETROS..."
